In [1]:
%run sim_env_setup.ipynb
%run kinematics.ipynb

1.8369701987210297e-16 3.0
1.5707963267948968 -3.141592653589793
0.6425397063779064 0.034418478095045946
-1.865174681370263e-14 2.7977620220553945e-14


In [2]:
import numpy as np
# Run this command to disable the motors
p.setJointMotorControlArray(scara, [1, 2], p.VELOCITY_CONTROL, forces=[0, 0])

In [3]:
p.setJointMotorControl2(scara, 1, p.TORQUE_CONTROL, force=-50)

for _ in range(1000):
    p.stepSimulation()
    
p.getJointState(scara, 1)

(-0.04443325490477883,
 -0.04351272342656368,
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 0.0)

In [4]:
# Read position at every time step and use proporitonal control to reach the position
dt = 0.001
p_gain = 2
d_gain = 5
error = 0
error_old = 0
desired_pos = 1.5
for _ in range(100000):
    pos, _, _, _ = p.getJointState(scara,1)
    error = desired_pos - pos
    error_d = (error - error_old)/dt
    control_force = p_gain * error + d_gain * error_d
    p.setJointMotorControl2(scara, 1, p.TORQUE_CONTROL, force=control_force)
    p.stepSimulation()    

In [5]:
# Read position at every time step and use proporitonal control to reach the position
dt = 0.001
p_gain = 2
d_gain = 5
error = 0
error_old = 0
desired_pos = np.array([1.5, 1.5])
for _ in range(100000):
    pos1, _, _, _ = p.getJointState(scara,1)
    pos2, _, _, _ = p.getJointState(scara,2)
    pos = np.array([pos1, pos2])
    error = desired_pos - pos
    error_d = (error - error_old)/dt
    control_force = p_gain * error + d_gain * error_d
    p.setJointMotorControlArray(scara, [1,2], p.TORQUE_CONTROL, forces=control_force)
    p.stepSimulation()

In [6]:
def control(theta1, theta2):
    # Read position at every time step and use proporitonal control to reach the position
    dt = 0.001
    p_gain = 2
    d_gain = 5
    error = 0
    error_old = 0
    desired_pos = np.array([theta1, theta2])
    for _ in range(100000):
        pos1, _, _, _ = p.getJointState(scara,1)
        pos2, _, _, _ = p.getJointState(scara,2)
        pos = np.array([pos1, pos2])
        error = desired_pos - pos
        error_d = (error - error_old)/dt
        control_force = p_gain * error + d_gain * error_d
        return control_force
        
        

In [9]:
# Kinematics for serial-2R

p1 = np.array([1.0, 0.5])
p2 = np.array([0.5, 1.0])
pt = p1

[theta1, theta2] = inverse_kinematics(pt[0], pt[1], 1, 1)

for _ in range(10000):
    control_force = control(theta1, theta2)
    p.setJointMotorControlArray(scara, [1,2], p.TORQUE_CONTROL, forces=control_force)
    p.stepSimulation()


In [14]:
p.disconnect()